<a href="https://colab.research.google.com/github/Aravind-11/IITM_Saastra/blob/main/Single_agent_with_obstacle_following.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import gym

class Football:  # The class encapsulating the environment
    '''
    Actions [0 : Stand, 1 : Up, 2 : Right, 3 : Down, 4 : Left]
    These are the representing no.s for the mentioned actions
    '''

    def __init__(self, length=30, width=30, goalPositions=[29, 15]):
        
        # The player start at random locations
        
        self.pA=[np.random.randint(length), np.random.randint(length)] 
        self.pO=[6,8]
            
        
        self.h = length   # Length of the Football Pitch    
        self.w = width    # Width of the Football Pitch
        
        self.goalPositions = np.array(goalPositions)   # This means that the middle 4 positions at the right and left are the goals
        
     
        
        self.reward = 0                            # Initially the reward is 0
        
        self.observation=np.random.rand(6,)
        self.done = bool(0)                          # This stores whether the game needs to be restart with new position (in the case of a goal)

    def reset(self):
        self.done = bool(0)
        self.reward = 0
        
        self.pA = np.array([np.random.randint(self.h), np.random.randint(self.h)])
        self.pO=[6,8]
        
        return np.array((*self.pA,(15-self.pA[0]),(15-self.pA[1]),*self.pO)).astype(np.float32)

    def step(self, action):
        if self.done == bool(1):
          self.reset()
        self.move(first, action)                   # We chose the first player at random
        if self.done == bool(1):
          return self.observation, self.reward, self.done
        if not done:
            self.current_player_num = (self.current_player_num + 1) % 2   
        return self.observation,self.reward, self.done

    def move(self, player, action):
        opponent = 1 - player
        
        newPosition = self.pA + self.actionToMove(action)
        
        if self.ballOwner is player and self.isInGoal(*newPosition) >= 0:
            self.done = bool(1)
            return 1 - self.isInGoal(*newPosition)
        # If it's in the board
        elif self.isInBoard(*newPosition):
            self.positions[player] = newPosition
        if(self.ballOwner!=0):
          self.reward=-1
        return -1

    def actionToMove(self, action):
        switcher = {
            0: [0, 0],
            1: [0, 1],
            2: [1, 0],
            3: [0, -1],
            4: [-1, 0],
        }
        return switcher.get(action)

    def isInGoal(self, x, y):
        g1, g2 = self.goalPositions
        if (g1 <= y <= g2):
            if x == 0:
                self.done = bool(1)
                self.reward = -20 # if the ball reaches the right goal post, then the rewards shall be -1
                return 1 
            elif x == (self.w-1):
                self.done = bool(1)
                self.reward = 20 # if the ball reaches the right goal post, then the rewards shall be 1
                return 0
        return -1

    def isInBoard(self, x, y):
        if(x<0 or x>29):
          return 0
        if(y<0 or y>29):
          return 0 
        return 1
        

    #def choosePlayer(self):
    #    return np.random.randint(0, 2)
    def render(self,mode="human"):
        

        board = ''
        for y in range(self.h)[::-1]:
            for x in range(self.w):
                if ([x, y] == self.pA).all():
                    board += 'A' 
                elif([x,y]==self.pO):
                  board+='O'
                else:
                    board += '-'
            board += '\n'

        print(board)

class modf_football(Football,gym.Env):
  def __init__(self, length=30, width=30, goalPositions=[29, 15]):
    super().__init__()
    self.observation_space=gym.spaces.Box(low=-30, high=30,
                                        shape=(6,), dtype=np.float32)
    self.reward=0
    self.action_space=gym.spaces.Discrete(5)
    self.name='Football'
    self.current_player_num=0
    self.observation=np.random.rand(6,)
    self.pA = np.array([np.random.randint(self.h), np.random.randint(self.h)])
  #modifying the step and move function to get the updated reward system
  def step(self, action):
        #print('action',action)
        if self.done == bool(1):
          self.reset()
        
        self.move(action)                   # We chose the first player at random
        if self.done == bool(1):
          return self.observation, self.reward, self.done,{}
        if not self.done:
            self.current_player_num = 0
        return self.observation,self.reward, self.done,{}
  
  def move(self, action):
        
        newPosition = self.pA + self.actionToMove(action)

        if (self.pO[0]>self.pA[0]):
          self.pO[0]-=1
        elif (self.pO[0]<self.pA[0]):
          self.pO[0]+=1
        if (self.pO[1]>self.pA[1]):
          self.pO[1]-=1
        elif (self.pO[1]<self.pA[1]):
          self.pO[1]+=1
        
        self.pO[0]=max(self.pO[0],0)
        self.pO[0]=min(self.pO[0],29)
        self.pO[1]=max(self.pO[1],0)
        self.pO[1]=min(self.pO[1],29)

        if self.isInGoal(*newPosition) >= 0:
            self.done = bool(1)
            return 1 - self.isInGoal(*newPosition)
        # If it's in the board
        elif self.isInBoard(*newPosition):
            self.reward = -0.1 * ((((abs(newPosition[0]-self.goalPositions[0]))+(abs(newPosition[1]-self.goalPositions[1]))))) +0.01*(abs(newPosition[0]-self.pO[0])+abs(newPosition[1]-self.pO[1]))
            self.pA = newPosition
        
        self.observation=np.array((*self.pA,(15-self.pA[0]),(15-self.pA[1]),*self.pO)).astype(np.float32)
        return -1
  def seed():
      return 0 
  def metadata(x):
      return 0 
  def legal_actions(self):
    return gym.spaces.Discrete(5)
  def close(self):
    pass

In [ ]:
env=modf_football(Football,gym.Env)

In [ ]:
print("Observation space:", env.observation_space)
print("Shape:", env.observation_space.shape)
# Discrete(2) means that there is two discrete actions
print("Action space:", env.action_space)

# The reset method is called at the beginning of an episode
obs = env.reset()
# Sample a random action
action = env.action_space.sample()
print("Sampled action:", action)
obs, reward, done, info = env.step(action)
# Note the obs is a numpy array
# info is an empty dict for now but can contain any debugging info
# reward is a scalar
print(obs, reward, done, info)

Observation space: Box(-30.0, 30.0, (6,), float32)
Shape: (6,)
Action space: Discrete(5)
Sampled action: 2
[ 7.  5.  8. 10.  6.  7.] -3.1700000000000004 False {}


In [ ]:
# Stable Baselines only supports tensorflow 1.x for now
%tensorflow_version 1.x
!pip install stable-baselines[mpi]==2.10.0

TensorFlow 1.x selected.
     |████████████████████████████████| 248 kB 16.0 MB/s 
  Attempting uninstall: stable-baselines
    Found existing installation: stable-baselines 2.2.1
    Uninstalling stable-baselines-2.2.1:
      Successfully uninstalled stable-baselines-2.2.1


In [ ]:
from stable_baselines.common.env_checker import check_env

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [ ]:
check_env(env, warn=True)

In [ ]:
from stable_baselines import DQN, PPO2, A2C, ACKTR
from stable_baselines.common.cmd_util import make_vec_env

# Instantiate the env
#env = GoLeftEnv(grid_size=10)
# wrap it
env = make_vec_env(lambda: env, n_envs=1)

In [ ]:

# Train the agent
model = PPO2('MlpPolicy', env, verbose=1).learn(5000)

--------------------------------------
| approxkl           | 0.00018498908 |
| clipfrac           | 0.0           |
| explained_variance | -0.00105      |
| fps                | 372           |
| n_updates          | 1             |
| policy_entropy     | 1.6092254     |
| policy_loss        | -0.0017238527 |
| serial_timesteps   | 128           |
| time_elapsed       | 1.67e-05      |
| total_timesteps    | 128           |
| value_loss         | 1350.0603     |
--------------------------------------
---------------------------------------
| approxkl           | 0.0001224053   |
| clipfrac           | 0.0            |
| explained_variance | 0.0185         |
| fps                | 921            |
| n_updates          | 2              |
| policy_entropy     | 1.6085753      |
| policy_loss        | -0.00037437375 |
| serial_timesteps   | 256            |
| time_elapsed       | 0.346          |
| total_timesteps    | 256            |
| value_loss         | 1200.3132      |
-------------

In [ ]:
obs = env.reset()
n_steps = 100
for step in range(n_steps):
  action, _ = model.predict(obs, deterministic=True)
  print("Step {}".format(step + 1))
  print("Action: ", action)
  obs, reward, done, info = env.step(action)
  print('obs=', obs, 'reward=', reward, 'done=', done)
  env.render(mode='console')
  if done:
    # Note that the VecEnv resets automatically
    # when a done signal is encountered
    print("Goal reached!", "reward=", reward)
    break

Step 1
Action:  [2]
obs= [[ 4. 16. 11. -1.  5.  9.]] reward= [-2.52] done= [False]
------------------------------
------------------------------
------------------------------
------------------------------
------------------------------
------------------------------
------------------------------
------------------------------
------------------------------
------------------------------
------------------------------
------------------------------
------------------------------
----A-------------------------
------------------------------
------------------------------
------------------------------
------------------------------
------------------------------
------------------------------
-----O------------------------
------------------------------
------------------------------
------------------------------
------------------------------
------------------------------
------------------------------
------------------------------
------------------------------
------------------

In [ ]:
model.save("ppo2_single_agent_moving_obstacle")